In [ ]:
import os
import pandas as pd
import numpy as np
import config
import pickle
from data_setup import ZRI_format
#Select path to credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=config.GOOGLE_APPLICATION_CREDENTIALS

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery --use_rest_api hud_crosswalk
SELECT *
FROM `bigquery-public-data.hud_zipcode_crosswalk.zip_to_county` 

In [ ]:
%%bigquery --use_rest_api crime
SELECT *
FROM `high-empire-220313.Crime.Crime_data` 

In [ ]:
ZRI_MF = pd.read_pickle('./pickles/ZRI_filtered.p')

In [ ]:
%%bigquery --use_rest_api FIPS
SELECT area_name, state_fips_code, county_fips_code
FROM `bigquery-public-data.census_utility.fips_codes_all` 

In [ ]:
crime

In [ ]:
crime = crime.assign(State = crime.county_name.apply(lambda x: abbrev_us_state[x[-2:]]),
                    county = crime.county_name.apply(lambda x: x[:-4]))
crime = crime.merge(FIPS[['area_name','state_fips_code']], how = 'left', left_on = 'State',right_on = 'area_name' )
crime = crime.merge(FIPS[['area_name','state_fips_code','county_fips_code']],how ='left',
                   left_on = ['county','state_fips_code'],right_on = ['area_name','state_fips_code'])

In [ ]:
crime.to_pickle('./pickles/crime.p',protocol=4)

In [ ]:
ZRI = ZRI_format(ZRI_MF, window_size = 6, future_time = 36)

In [ ]:
zri_zips = ZRI.ZipCode.unique()
dominant_county_zip = {zip_code : hud_crosswalk.loc[hud_crosswalk[hud_crosswalk['zip_code'] == zip_code]['total_ratio'].idxmax(axis = 0),'county_geoid' ] for zip_code in zri_zips}

In [ ]:
#hud_crosswalk = hud_crosswalk.assign(total_ratio = hud_crosswalk['total_ratio'].astype(float))

In [ ]:
ZRI = ZRI.assign(dominant_county = ZRI.ZipCode.apply(lambda x: dominant_county_zip[x]))

In [ ]:
ZRI = ZRI.merge(crime[['crime_rate_per_100000','county_fips_code']],how = 'left',
          left_on = 'dominant_county',right_on = 'county_fips_code').drop('county_fips_code',axis = 1)

In [ ]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
with open('./pickles/dominant_county_zip.p','wb') as f:
    pickle.dump(dominant_county_zip,f)